# Рубежный контроль №2
**Бурашников Владимир Владимирович, группа ИУ5-22М.**
Вариант №1.

## Задание
Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета. Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.

Необходимо сформировать признаки на основе `CountVectorizer` или `TfidfVectorizer`.

В качестве классификаторов необходимо использовать один из классификаторов, не относящихся к наивным Байесовским методам (например, `LogisticRegression`), а также Multinomial Naive Bayes (MNB), Complement Naive Bayes (CNB), Bernoulli Naive Bayes.
Для каждого метода необходимо оценить качество классификации с помощью хотя бы одной метрики качества классификации (например, `accuracy`).

Сделайте выводы о том, какой классификатор осуществляет более качественную классификацию на Вашем наборе данных.

## Решение

### Загрузка и предобработка данных

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [23]:
#rcv_train = fetch_rcv1(subset='train')
#rcv_test = fetch_rcv1(subset='test')
df = pd.read_json('./news.json', lines=True)

In [24]:
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [87]:
headline = np.array(df['headline'])
category = np.array(df['category'])
# build train and test datasets

# Train/test splitting for 41 categories of news
from sklearn.model_selection import train_test_split    
headline_train, headline_test, category_train, category_test = train_test_split(headline, category, test_size=0.2, random_state=41)

In [88]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## Build Bag-Of-Words on train phrases
cv = CountVectorizer(stop_words='english',max_features=10000)
cv_train_features = cv.fit_transform(headline_train)

In [89]:
# build TFIDF features on train reviews
tv = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(headline_train)

In [91]:
cv_test_features = cv.transform(headline_test)
tv_test_features = tv.transform(headline_test)

### Обучение моделей

In [110]:
from sklearn.metrics import accuracy_score

In [113]:
from sklearn import metrics
import numpy as np

def accuracy(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    classifier.fit(train_features, train_labels)
    print('Accuracy:', metrics.accuracy_score(test_labels, classifier.predict(test_features)))    


In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB

In [115]:
lr = LogisticRegression(solver='lbfgs',penalty='l2', max_iter=100, C=1,multi_class='auto')

lr_accuracy = accuracy(classifier=lr,train_features=cv_train_features, train_labels=category_train,
                       test_features=cv_test_features, test_labels=category_test)

Accuracy: 0.5750665903263549


/home/vladimir/PycharmProjects/giis_lab1/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [116]:
mu = MultinomialNB()

mu_accuracy = accuracy(classifier=mu,train_features=cv_train_features, train_labels=category_train,
                       test_features=cv_test_features, test_labels=category_test)

Accuracy: 0.5516915187573125


In [117]:
co = ComplementNB()

co_accuracy = accuracy(classifier=co,train_features=cv_train_features, train_labels=category_train,
                       test_features=cv_test_features, test_labels=category_test)

Accuracy: 0.5401906848223843


In [118]:
be = BernoulliNB()

be_accuracy = accuracy(classifier=be,train_features=cv_train_features, train_labels=category_train,
                       test_features=cv_test_features, test_labels=category_test)

Accuracy: 0.5414104702397252


### Вывод
Метод LogisticRegression, лучше всего решает поставленную задачу многоклассовой классификации. Но значительно больше затрачивает ресурсов и времени.